In [3]:
#importing the necessary packages

import tensorflow as tf
from tensorflow import keras
tf.__version__

'2.10.0'

In [4]:
model = keras.models.load_model('xception_v4_1_10_0.998.h5')

2022-12-19 23:05:07.362844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 23:05:07.427472: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [21]:
img = load_img('capsicum.jpeg', target_size=(299,299))

In [22]:
import numpy as np

In [23]:
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)


In [24]:
preds = model.predict(X)

1/1 [==============================] - 2s 2s/step


In [25]:
classes = [
    'Bean',
    'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']


In [26]:
dict(zip(classes, preds[0]))

{'Bean': -0.28128734,
 'Bitter_Gourd': -2.9696662,
 'Bottle_Gourd': 0.06928816,
 'Brinjal': -3.4746149,
 'Broccoli': -1.3456322,
 'Cabbage': -6.085357,
 'Capsicum': 15.849345,
 'Carrot': -4.867174,
 'Cauliflower': -3.699632,
 'Cucumber': -10.427808,
 'Papaya': 3.4041345,
 'Potato': -12.156144,
 'Pumpkin': -15.335313,
 'Radish': -7.034167,
 'Tomato': 4.336424}

## Convert Keras to TFLite

In [27]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('vegclass-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpv90a5ebq/assets


INFO:tensorflow:Assets written to: /tmp/tmpv90a5ebq/assets
2022-12-19 23:49:11.862774: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-19 23:49:11.862918: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-19 23:49:12.016763: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpv90a5ebq
2022-12-19 23:49:12.166348: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-19 23:49:12.166466: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpv90a5ebq
2022-12-19 23:49:12.259534: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-12-19 23:49:12.424859: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is n

In [1]:
import tensorflow.lite as tflite

2022-12-20 00:19:13.709237: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
interpreter = tflite.Interpreter(model_path='vegclass-model.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
input_index = interpreter.get_input_details()[0]['index']

In [4]:
output_index = interpreter.get_output_details()[0]['index']

In [6]:
interpreter.set_tensor(input_index, X)

In [ ]:
interpreter.invoke()

In [41]:
preds = interpreter.get_tensor(output_index)

In [42]:
classes = [
    'Bean',
    'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']
dict(zip(classes, preds[0]))

{'Bean': -0.28129038,
 'Bitter_Gourd': -2.969659,
 'Bottle_Gourd': 0.06928937,
 'Brinjal': -3.4746141,
 'Broccoli': -1.3456353,
 'Cabbage': -6.0853596,
 'Capsicum': 15.849351,
 'Carrot': -4.8671756,
 'Cauliflower': -3.699637,
 'Cucumber': -10.42781,
 'Papaya': 3.404147,
 'Potato': -12.156142,
 'Pumpkin': -15.335322,
 'Radish': -7.0341763,
 'Tomato': 4.33642}

### Removing the TF Dependency

In [14]:
!python3 -m pip install tflite-runtime

In [1]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
interpreter = tflite.Interpreter(model_path='vegclass-model.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [4]:
X = preprocessor.from_path('capsicum.jpeg')

In [5]:
# load_img('capsicum.jpeg')

In [6]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [7]:
classes = [
    'Bean',
    'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']
dict(zip(classes, preds[0]))

{'Bean': -0.28129038,
 'Bitter_Gourd': -2.969659,
 'Bottle_Gourd': 0.06928937,
 'Brinjal': -3.4746141,
 'Broccoli': -1.3456353,
 'Cabbage': -6.0853596,
 'Capsicum': 15.849351,
 'Carrot': -4.8671756,
 'Cauliflower': -3.699637,
 'Cucumber': -10.42781,
 'Papaya': 3.404147,
 'Potato': -12.156142,
 'Pumpkin': -15.335322,
 'Radish': -7.0341763,
 'Tomato': 4.33642}